TODO 1:
    * create environment: done
    * install SIAC within the environment: done
    * modify files to use current GDAL version: ongoing
    
- 'raster_boundary.py'
    'import osr' -> 'from osgeo import osr, ogr, gdal
    'import ogr' -> ''
    'import gdal' -> ''
    
- 'reproject.py'
    'import gdal' -> ''
    'from osgeo import osr' -> 'from osgeo import osr, gdal'
    
- 'get_MCD43B.py'
    'import gdal' -> 'from osgeo import gdal'

- 'modis_tile_cal.py'
    'import osr' -> from osgeo import osr, ogr, gdal,__version__ as gdalversion'
    'import ogr' -> ''
    'import gdal' -> ''
    L39: 
    if int(gdalversion[0])>3:
        (ul_lon, ul_lat, ulz) = tx.TransformPoint(geo_t[3], geo_t[0])
        (lr_lon, lr_lat, lrz) = tx.TransformPoint(geo_t[3] + geo_t[5] * y_size, geo_t[0] + geo_t[1] * x_size)
        (ll_lon, ll_lat, llz) = tx.TransformPoint(geo_t[3] + geo_t[5] * y_size, geo_t[0])
        (ur_lon, ur_lat, urz) = tx.TransformPoint(geo_t[3], geo_t[0] + geo_t[1] * x_size)
    else:
        (ul_lon, ul_lat, ulz ) = tx.TransformPoint( geo_t[0], geo_t[3])
        (lr_lon, lr_lat, lrz ) = tx.TransformPoint( geo_t[0] + geo_t[1] * x_size, geo_t[3] + geo_t[5] * y_size )
        (ll_lon, ll_lat, llz ) = tx.TransformPoint( geo_t[0] , geo_t[3] + geo_t[5] * y_size )
        (ur_lon, ur_lat, urz ) = tx.TransformPoint( geo_t[0] + geo_t[1] * x_size, geo_t[3])    
    L85:
    if int(gdalversion[0])>3:
        ho, vo, z = tx.TransformPoint(lat, lon)  # still use the function instead of using the equation....
    else:
        ho,vo,z = tx.TransformPoint(lon, lat)# still use the function instead of using the equation....
    
- 's2_preprocessing.py'
    'import gdal' -> 'from osgeo import gdal'
    
- 's2_angle.py'
    'import ogr' -> 'from osgeo import ogr,gdal'
    'import gdal' -> ''
- 'SIAC_L8.py'    
    'import gdal' -> 'from osgeo import gdal'
- 'l8_angle.py'
    'import gdal' -> 'from osgeo import gdal'


In [1]:
1>=1

True

# 3 Setup
All the code for the individual modules are is located at https://github.com/multiply-org/. This can be used to setup the MULTIPLY framework on your own computing infrastructure. At present however no deployment setup (in the form of windows-setup-executables, or anaconda package’s) exist. While this is planned further intolater in the project, the focus at this stage is on testing the individual components themselves. Please let us know if you would prefer to install the software yourself on a dedicated computational framework, so that we can investigate how to facilitate this for you. 
In order to facilitate the testing of the framework itself, we have setup this Virtual Machine on Google Compute Engine, for testing purposes. 


## 3.1 Parameters
Here you can actually set the parameters for the run. 


### Define working_directory

In [2]:
working_directory_name = 'OVP_test_pynb36upd_20220717'

### 3.1.1 Define region of Interest
* **roi**: A region of interest, given as a Polygon in WKT format. You can use this tool ( https://arthur-e.github.io/Wicket/sandbox-gmaps3.html ) to easily get definitions of the regions you are interested in in WGS84 coordinates.
* **roi_grid**: The EPSG-code of the spatial reference system in which the roi is given. If it is set to 'none', it is assumed that the roi is given in WGS84 coordinates.
* **destination_grid**: The EPSG-code of the spatial reference system in which the output shall be given. If it is set to 'none', the platform will attempt to derive it from the roi_grid.
* **spatial_resolution**: The resolution the output data is supposed to have, must be a non-negative integer number. The resolution is given in meters and is the same for both dimensions.

In [3]:
roi = 'POLYGON(( 5.695 52.26,  5.695 52.25,  5.680 52.25,  5.680 52.26,  5.695 52.26 ))' #Speulderbos. Something goes wrong (possibly due to 'overlapping tiles')
roi = 'POLYGON ((5.163574 52.382529, 5.163574 52.529813, 5.493164 52.529813, 5.493164 52.382529, 5.163574 52.382529))' #OVP. Correct in MUI version
# roi = 'POLYGON ((5.163 52.38,  5.163 52.52,  5.493 52.52,  5.493 52.38,  5.163 52.38 ))'

# roi_grid = 'EPSG:4326'                            # WGS84
# destination_grid = 'EPSG:4326'                   # WGS84
spatial_resolution = 20 # in m

### 3.1.2 Define temporal frequency and period
* **start_time**: The start date of the period you are interested in, must be given in the format 'Year-Month-Day' as below.
* **end_time**: The end date of the period you are interested in, must be given in the format 'Year-Month-Day' as below.
* **time_step**: The temporal resolution the output is supposed to have. Data will be aggregated over the period denoted by this parameter. Must be a non-negative integer value. The unit is days.

In [4]:
# start_time_as_string = '2019-04-16'
# stop_time_as_string = '2019-04-17'

start_time_as_string = '2018-04-16'
stop_time_as_string = '2018-04-20'

time_step = 5 # in days


### 3.1.2 Variables 
* **variables**: The list of the biophysical variables that shall be derived. Please do not change this list, as the underlying forward model requires all of them. The parameters are as follows:
  * **n**: Structural parameter
  * **cab**: Leaf Chlorophyll Content, given in ug/cm²
  * **car**: Leaf Carotonoid Content, given in ug/cm²
  * **cb**: Leaf senescent material
  * **cw**: Leaf Water Content, given in cm
  * **cdm**: Leaf Dry Mass, given in g/cm²
  * **lai**: Effective Leaf Area Index, given in m²/m²
  * **ala**: Average Leaf Angle, given in degrees
  * **bsoil**: Soil Brightness Parameter
  * **psoil**: Soil Wetness Parameter
* **file_mask**: A file that can be used to explicitly state the region you are interested in. You can also use it to mask out single pixels within this region. If this is not 'none', the aforementioned parameters roi_grid, spatial_resolution, and destination_grid are not used.

**HINT**: The platform will perform faster the smaller your roi and the larger the spatial resolution is.

In [5]:
# variables = {'n', 'cab', 'car', 'cb', 'cw', 'cdm', 'lai', 'ala', 'bsoil', 'psoil'}
# file_mask = None

## 3.2 Load internal packages and auxiliary methods

In [6]:
import datetime
import glob
import os

## 3.2 Defining the additional interfaces

In [7]:
def create_dir(dir):
    try:
        if not os.path.exists(dir):
            os.makedirs(dir)
    except:
        print(dir)
    return

In [8]:
# import SIAC


In [9]:
def preprocess_s2(s2_l1c_dir: str, modis_dir: str, emus_dir: str, cams_dir: str, dem_dir: str, output_root_dir: str,
               roi: str):
    vrt_dem_file = glob.glob(dem_dir + '/' + '*.vrt')[0]
    processor_dir = '/home/jtimmer1/software/atmospheric_correction/SIAC'
    create_dir(output_root_dir)
    dirs = glob.glob(s2_l1c_dir + "/*")
    for directory in dirs:
        directory_parts = directory.split('/')
        product_name = f"{directory_parts[-1]}-ac"
        print(f'Start pre-processing S2 L1 data from {directory_parts[-2]}')
        output_dir = output_root_dir + '/' + product_name + '/'
        roi_str =  roi #"\'" + roi + "\'"
        
#         command = "PYTHONPATH=$PYTHONPATH:" + processor_dir + "/util python " + processor_dir + "/SIAC_S2.py" +\
#                     " -f " + directory + "/" +\
#                     " -m " + modis_dir + \
#                     " -e " + emus_dir + \
#                     " -c " + cams_dir + \
#                     " -d " + vrt_dem_file +\
#                     " -o " + "False" +\
#                     " -a " + roi_str
#         os.system(command)
        
        import SIAC
        import pdb
        pdb.set_trace()
        SIAC.SIAC_S2(s2_t=directory+"/", dem_vrt=vrt_dem_file, cams_dir=cams_dir, mcd43=modis_dir,
        vrt_dir=modis_dir, download_mcd43="False", emu_dir=emus_dir, aoi=roi_str) 

        print(f'Maintenance')
        create_dir(output_dir)
        cmd2 = "mv $(find " + directory + '/ -type f) ' + output_dir + '/'
        os.system(cmd2)

        cmd3 = "cp `readlink " + directory + "/metadata.xml` " + output_dir + "/metadata.xml"
        os.system(cmd3)
        cmd3 = "cp `readlink " + directory + "/MTD_MSIL1C.xml` " + output_dir + "/MTD_MSIL1C.xml"
        os.system(cmd3)
        paths_to_mtd_tl = glob.glob(os.path.join(directory, 'GRANULE/*/MTD_TL.xml'))                
        if len(paths_to_mtd_tl) > 0:
            cmd3 = "cp `readlink " + paths_to_mtd_tl[0] + "` " + output_dir + "/MTD_TL.xml"
        os.system(cmd3)
        print(f'Finished pre-processing S2 L1 data from {directory_parts[-2]}')

In [10]:
# import os
# import shutil
# def get_working_dir(dir_name: str) -> str:
#     working_dir = f'/datastore/working_dirs/{dir_name}'
#     if os.path.exists(working_dir):
#         shutil.rmtree(working_dir)
#     os.makedirs(working_dir)
#     return working_dir

# 4 Running MULTIPLY
Below the actual code is provided for running the MULTIPLY framework.
We start with setting earth data authentication. This is required to download the MODIS brdf descriptors which are required for the atmospheric correction of the Sentinel-2 data. You can get credentials when you register at https://urs.earthdata.nasa.gov/profile . Registration and use is free of cost. If you do not register, you can only use the MODIS data which has been downloaded in previous runs of the notebook by other users.
Also you will need to set up the data stores so that the data access component is working correctly and finds the pre-configured data stores. Both steps only need to be performed once.

### 4.1 Creating working directory
For this notebook, you will operate in your own working directory. All data you use will be copied here, all output will be written here.

In [11]:
start_time_as_datetime = datetime.datetime.strptime(start_time_as_string, '%Y-%m-%d')
stop_time_as_datetime = datetime.datetime.strptime(stop_time_as_string, '%Y-%m-%d')

time_step_as_time_delta = datetime.timedelta(days=time_step)

In [12]:
# Setup clean working directory
# working_dir = get_working_dir(name)

# use previous (non-empty) working directory
name = working_directory_name
working_dir = '/Data/test_user_16/' + name
working_dir = '/datastore/working_dirs/' + name

In [13]:
# print('Working directory is {}'.format(working_dir))
# priors_directory = '{}/priors'.format(working_dir)
# hres_state_dir = '{}/hresstate'.format(working_dir)
# state_directory = '{}/state'.format(working_dir)
# biophys_output = '{}/biophys'.format(working_dir)

s2_l1c_directory = '{}/s2'.format(working_dir)
modis_directory = '{}/modis'.format(working_dir)
cams_directory = '{}/cams'.format(working_dir)
sdrs_directory = '{}/sdrs'.format(working_dir)
emulators_directory = '{}/emulators'.format(working_dir)
dem_directory = '{}/dem'.format(working_dir)

# print(working_dir)

### 5.2.1 Download Static Data

In [14]:
# # Setting up
cursor = start_time_as_datetime
# previous_inference_state = None #'none'
# updated_inference_state = 'none'
# one_day_step = datetime.timedelta(days=1)
# preprocess_only_region_of_interest = True

date_as_string = datetime.datetime.strftime(cursor, '%Y-%m-%d')

# cursor += time_step_as_time_delta
# cursor -= one_day_step
# if cursor > stop_time_as_datetime:
#     cursor = stop_time_as_datetime
# next_date_as_string = datetime.datetime.strftime(cursor, '%Y-%m-%d')
# cursor += one_day_step
# cursor_as_string = datetime.datetime.strftime(cursor, '%Y-%m-%d')

s2_l1c_directory_for_date = '{}/{}'.format(s2_l1c_directory, date_as_string)
modis_directory_for_date = '{}/{}'.format(modis_directory, date_as_string)
cams_directory_for_date = '{}/{}'.format(cams_directory, date_as_string)
sdrs_directory_for_date = '{}/{}'.format(sdrs_directory, date_as_string)
# priors_directory_for_date = '{}/{}/'.format(priors_directory, date_as_string)


### 6.1.2 Preprocessing

#### DEBUG

In [15]:
import pickle as pkl
import sklearn
import sys


In [16]:
import pdb


In [17]:
def Try_this_out():
    pythonversion = sys.version
    #read data 
    if int(pythonversion[2])==6:
        fid = open('/home/jtimmer1/software/atmospheric_correction/SIAC/data/sen2cloud_detector_v36.pkl', 'rb')
        cl = pkl.load(fid)
        print('loaded 36 version')
    else:
        fid = open('/home/jtimmer1/software/atmospheric_correction/SIAC/data/sen2cloud_detector_v39.pkl', 'rb')
        cl = pkl.load(fid)
        print('loaded 39 version')
    fid.close()
    
    # modify the sklearn-object for version 0.20.3 to the current version (1.1.1)
    if int(sklearn.__version__[0])>=1:
        cl.set_params(class_weight=None)  # TO SOLVE 'LGBMClassifier' object has no attribute 'class_weight'
        cl.set_params(importance_type=None) # TO SOLVE 'LGBMClassifier' object has no attribute 'importance_type'
        cl.set_params(fitted_=True)
    
    # modify the original pkl file to update the sklearn-object for version 0.20.3 to the current version (1.1.1)
    if int(sklearn.__version__[0])>=1:
        fid = open('/home/jtimmer1/software/atmospheric_correction/SIAC/data/sen2cloud_detector_v39.pkl','wb')
        pkl.dump(cl,fid)
        fid.close()
        print('Updated 39 version')
    else:
        print('Left 36 version unaltered')
            
    #############################
    ## Test prediction 
    #############################
    # open toas
    fid = open('/home/jtimmer1/toas.pkl','rb')
    toas = pkl.load(fid)
    fid.close()
    
#     import pdb
#     pdb.set_trace()
#     cl.__sklearn_is_fitted__ = True
    BOAs = cl.predict_proba(toas)
    return BOAs

In [18]:
# cl = Try_this_out()

# print(cl)

[[0.97822609 0.02177391]
 [0.96400097 0.03599903]
 [0.9381087  0.0618913 ]
 ...
 [0.03016242 0.96983758]
 [0.01373966 0.98626034]
 [0.04775515 0.95224485]]
 
[[0.97822609 0.02177391]
 [0.96400097 0.03599903]
 [0.9381087  0.0618913 ]
 ...
 [0.03016242 0.96983758]
 [0.01373966 0.98626034]
 [0.04775515 0.95224485]]

#### Run preprocess


- pre_process is giving an error, most probably because when reading one of the bands using '_do_chunck(1)' in 'the_correction: L417', NAN values are returned...
This continue with  the investigation... check self._toa_bands (when interrupting the code execution and going up to the specified line

In [19]:
preprocess_s2(s2_l1c_directory_for_date, modis_directory_for_date, emulators_directory, cams_directory_for_date, 
          dem_directory, sdrs_directory_for_date, roi)

Start pre-processing S2 L1 data from 2018-04-16
> /tmp/ipykernel_18264/4076071717.py(27)preprocess_s2()
     25         import pdb
     26         pdb.set_trace()
---> 27         SIAC.SIAC_S2(s2_t=directory+"/", dem_vrt=vrt_dem_file, cams_dir=cams_dir, mcd43=modis_dir,
     28         vrt_dir=modis_dir, download_mcd43="False", emu_dir=emus_dir, aoi=roi_str) 
     29 



ipdb>  c


2022-07-28 16:22:36,808 - SIAC-V2.2.0 - INFO - Starting SIAC
INFO:ComponentProgress:0
2022-07-28 16:22:36,823 - SIAC-V2.2.0 - INFO - Preprocessing for S2B_OPER_MSI_L1C_TL_MPS__20180416T111231_A005797_T31UFU_N02.06
2022-07-28 16:22:36,824 - SIAC-V2.2.0 - INFO - Doing per pixel angle resampling.
2022-07-28 16:22:36,974 - SIAC-V2.2.0 - INFO - getting angles from 13 processes
2022-07-28 16:22:36,975 - SIAC-V2.2.0 - INFO - getting angle for /datastore/working_dirs/OVP_test_pynb36upd_20220717/s2/2018-04-16/31/U/FU/2018/4/16/0/B01.jp2, /datastore/working_dirs/OVP_test_pynb36upd_20220717/s2/2018-04-16/31/U/FU/2018/4/16/0/angles/VAA_VZA_B01.tif, and /datastore/working_dirs/OVP_test_pynb36upd_20220717/s2/2018-04-16/31/U/FU/2018/4/16/0/qi/MSK_DETFOO_B01.gml
2022-07-28 16:22:37,893 - SIAC-V2.2.0 - INFO - getting angle for /datastore/working_dirs/OVP_test_pynb36upd_20220717/s2/2018-04-16/31/U/FU/2018/4/16/0/B02.jp2, /datastore/working_dirs/OVP_test_pynb36upd_20220717/s2/2018-04-16/31/U/FU/2018/4/16

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


2022-07-28 16:32:42,656 - MultiGrid solver - INFO - CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
2022-07-28 16:32:42,658 - MultiGrid solver - INFO - Iterations: 3
2022-07-28 16:32:42,659 - MultiGrid solver - INFO - Function calls: 19
INFO:ComponentProgress:70
INFO:ComponentProgress:70
INFO:ComponentProgress:70
INFO:ComponentProgress:70
INFO:ComponentProgress:70
INFO:ComponentProgress:70
2022-07-28 16:32:42,664 - MultiGrid solver - INFO - Optimizing at grid level 2


...

2022-07-28 16:32:49,847 - MultiGrid solver - INFO - CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
2022-07-28 16:32:49,849 - MultiGrid solver - INFO - Iterations: 7
2022-07-28 16:32:49,850 - MultiGrid solver - INFO - Function calls: 20
INFO:ComponentProgress:78
INFO:ComponentProgress:78
INFO:ComponentProgress:78
INFO:ComponentProgress:78
INFO:ComponentProgress:78
INFO:ComponentProgress:78
2022-07-28 16:32:50,334 - SIAC-V2.2.0 - INFO - Finished retrieval and saving them into local files.
2022-07-28 16:32:52,917 - SIAC-V2.2.0 - INFO - Set AOI.
2022-07-28 16:32:52,921 - SIAC-V2.2.0 - INFO - Slice TOA bands based on AOI.
2022-07-28 16:32:54,811 - SIAC-V2.2.0 - INFO - Parsing angles.
2022-07-28 16:32:55,179 - SIAC-V2.2.0 - INFO - Parsing auxs.
2022-07-28 16:32:55,194 - SIAC-V2.2.0 - INFO - Parsing atmo parameters.
2022-07-28 16:33:03,635 - SIAC-V2.2.0 - INFO - Loading emus.
2022-07-28 16:33:10,499 - SIAC-V2.2.0 - INFO - Get correction coefficients.
2022-07-28 16:33:14,127 - SIAC-V2.2.0 - I

Maintenance


mv: will not overwrite just-created ‘/datastore/working_dirs/OVP_test_pynb36upd_20220717/sdrs/2018-04-16/31-ac/MSK_DETFOO_B01.gfs’ with ‘/datastore/working_dirs/OVP_test_pynb36upd_20220717/s2/2018-04-16/31/U/FU/2018/4/16/1/qi/MSK_DETFOO_B01.gfs’
mv: will not overwrite just-created ‘/datastore/working_dirs/OVP_test_pynb36upd_20220717/sdrs/2018-04-16/31-ac/MSK_DETFOO_B02.gfs’ with ‘/datastore/working_dirs/OVP_test_pynb36upd_20220717/s2/2018-04-16/31/U/FU/2018/4/16/1/qi/MSK_DETFOO_B02.gfs’
mv: will not overwrite just-created ‘/datastore/working_dirs/OVP_test_pynb36upd_20220717/sdrs/2018-04-16/31-ac/MSK_DETFOO_B03.gfs’ with ‘/datastore/working_dirs/OVP_test_pynb36upd_20220717/s2/2018-04-16/31/U/FU/2018/4/16/1/qi/MSK_DETFOO_B03.gfs’
mv: will not overwrite just-created ‘/datastore/working_dirs/OVP_test_pynb36upd_20220717/sdrs/2018-04-16/31-ac/MSK_DETFOO_B04.gfs’ with ‘/datastore/working_dirs/OVP_test_pynb36upd_20220717/s2/2018-04-16/31/U/FU/2018/4/16/1/qi/MSK_DETFOO_B04.gfs’
mv: will not ove

Finished pre-processing S2 L1 data from 2018-04-16


cp: missing destination file operand after ‘/datastore/working_dirs/OVP_test_pynb36upd_20220717/sdrs/2018-04-16/31-ac//MTD_MSIL1C.xml’
Try 'cp --help' for more information.


In [20]:
to_inv = np.nansum([sparse.diags((self.obs_unc[0]).ravel()).toarray(), sparse.diags((self.prior_uncs[0]**-2).ravel()).toarray(), self.gamma**2 * dtd.toarray()], axis = 0)
to_inv = np.nansum([sparse.diags((self.obs_unc[1]).ravel()).toarray(), sparse.diags((self.prior_uncs[1]**-2).ravel()).toarray(), self.gamma**2 * dtd.toarray()], axis = 0)

NameError: name 'np' is not defined

In [ ]:
import numpy as np
from scipy import optimize, interpolate, sparse
def compose_dtd(nx, ny):
    ns = nx*ny
    n = int(np.sqrt(ns))
    d1 = 2 * np.ones(ns)
    d1[ny-1::ny] = 1
    d1[0::ny] = 1
    d2 = np.ones(ns) * -1
    d2[ny-1::ny] = 0
    d3 = 2 * np.ones(ns)
    d3[:ny] = 1
    d3[ns-ny:] = 1
    d4 = np.ones(ns) * -1
    dtdx = sparse.spdiags([d1, d2[::-1], d2], [0, 1, -1], ns, ns)
    dtdy = sparse.spdiags([d3, d4, d4], [0, ny, -ny], ns, ns)
    try:
        dtd = dtdx + dtdy
    except:
        dtd = sparse.csr_matrix( dtdx.toarray() + dtdy.toarray() )
        
    return dtd, dtdx, dtdy


In [ ]:
nx=1
ny=414
II,II1,II2 = compose_dtd(nx,ny)

In [ ]:
sparse.linalg.inv(sparse.csc_matrix(II.toarray())).diagonal()


In [ ]:
2022-07-22 15:45:04,933 - MultiGrid solver - INFO - Total 4 level of grids are going to be used.
INFO:ComponentProgress:68
INFO:ComponentProgress:68
INFO:ComponentProgress:68
INFO:ComponentProgress:68
INFO:ComponentProgress:68
INFO:ComponentProgress:68
2022-07-22 15:45:04,940 - MultiGrid solver - INFO - Optimizing at grid level 1

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

2022-07-22 15:45:06,465 - MultiGrid solver - INFO - b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
2022-07-22 15:45:06,469 - MultiGrid solver - INFO - Iterations: 3
2022-07-22 15:45:06,469 - MultiGrid solver - INFO - Function calls: 21
INFO:ComponentProgress:70
INFO:ComponentProgress:70
INFO:ComponentProgress:70
INFO:ComponentProgress:70
INFO:ComponentProgress:70
INFO:ComponentProgress:70
2022-07-22 15:45:06,472 - MultiGrid solver - INFO - Optimizing at grid level 2

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

2022-07-22 15:45:10,591 - MultiGrid solver - INFO - b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
2022-07-22 15:45:10,594 - MultiGrid solver - INFO - Iterations: 6
2022-07-22 15:45:10,595 - MultiGrid solver - INFO - Function calls: 80
INFO:ComponentProgress:73
INFO:ComponentProgress:73
INFO:ComponentProgress:73
INFO:ComponentProgress:73
INFO:ComponentProgress:73
INFO:ComponentProgress:73
2022-07-22 15:45:10,599 - MultiGrid solver - INFO - Optimizing at grid level 3

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

2022-07-22 15:45:12,050 - MultiGrid solver - INFO - b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
2022-07-22 15:45:12,051 - MultiGrid solver - INFO - Iterations: 4
2022-07-22 15:45:12,052 - MultiGrid solver - INFO - Function calls: 20
INFO:ComponentProgress:75
INFO:ComponentProgress:75
INFO:ComponentProgress:75
INFO:ComponentProgress:75
INFO:ComponentProgress:75
INFO:ComponentProgress:75
2022-07-22 15:45:12,056 - MultiGrid solver - INFO - Optimizing at grid level 4

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

2022-07-22 15:45:16,761 - MultiGrid solver - INFO - b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
2022-07-22 15:45:16,763 - MultiGrid solver - INFO - Iterations: 8
2022-07-22 15:45:16,764 - MultiGrid solver - INFO - Function calls: 43


In [ ]:
i=2
ret = rets[i]
S2A_OPER_MSI_L1C_TL_MPS__20180418T125356_A014734_T31UFU_N02.06

In [ ]:
# INFO:ComponentProgress:20
# 2022-07-19 16:55:39,322 - SIAC-V2.2.0 - INFO - Starting atmospheric corretion for S2B_OPER_MSI_L1C_TL_MPS__20180416T111231_A005797_T31UFU_N02.06
# 2022-07-19 16:55:41,979 - SIAC-V2.2.0 - INFO - Set AOI.
# 2022-07-19 16:55:41,984 - SIAC-V2.2.0 - INFO - Get corresponding bands.
# 2022-07-19 16:55:41,985 - SIAC-V2.2.0 - INFO - Slice TOA bands based on AOI.
# 2022-07-19 16:55:43,212 - SIAC-V2.2.0 - INFO - Parsing angles.
# 2022-07-19 16:55:43,215 - SIAC-V2.2.0 - INFO - Mask bad pixels.
# ERROR 1: Passed object is neither of type string nor bytes
# ERROR 1: Passed object is neither of type string nor bytes
# 2022-07-19 16:55:43,780 - SIAC-V2.2.0 - INFO - No valid value is found for retrieval of atmospheric parameters and priors are stored.
# ERROR 1: Passed object is neither of type string nor bytes

fid = open('/home/jtimmer1/toas.pkl','wb')
pkl.dump(toas[:,mask].T,fid)
fid.close()

# the Contributing team
<img src="pics/Logos.png">
